<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Physical-model" data-toc-modified-id="Physical-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Physical model</a></span></li><li><span><a href="#Optimisation-probleme-formulation" data-toc-modified-id="Optimisation-probleme-formulation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Optimisation probleme formulation</a></span></li><li><span><a href="#Solve-Problem" data-toc-modified-id="Solve-Problem-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Solve Problem</a></span></li><li><span><a href="#Brut-Force-approach" data-toc-modified-id="Brut-Force-approach-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Brut Force approach</a></span></li></ul></div>

In [ ]:
#%matplotlib notebook
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from scipy import optimize
import ipywidgets as ipw
from matplotlib import cm
import ipywidgets as widgets
from scipy import integrate



# Crossing a river
This exemple deal with the case of a personne that want to cross a river in a minimum time. 

Probleme statement:
* the velocity on ground is 10 m/s
* the swiming velocity is 2 m/s
* the starting point and the ending point are fixed
* the width of the river is fixed

::::{admonition} Example :class: tip

Problem formualtion:

Objective: 

Variables: 

Constraints: 

Objective function: 

Resolution method: 
::::

## Where is the river : The landscape 
The river is an area wher the velocity is lower.

In [ ]:
def world_definition(x, y):
    v = 1 * np.ones_like(x)
    v[y <= 10] = 10
    v[y > 50] = 10
    return v


x = np.linspace(0, 100, 100)
y = np.linspace(0, 60, 150)
x_world, y_world = np.meshgrid(x, y, indexing="xy")
v_world = world_definition(x_world, y_world)

In [ ]:
plt.figure()
plt.contourf(x_world, y_world, v_world, 20, cmap=cm.jet)
plt.title("Velocity map")
plt.colorbar()
plt.show()

## Define the path

In [ ]:
path = np.array([[10.0, 5.0], [15.0, 10], [25, 50], [80, 55]])

plt.figure()
plt.contourf(x_world, y_world, v_world, 20, cmap=cm.jet)
plt.plot(path[:, 0], path[:, 1])
plt.title("Velocity map")
plt.colorbar()

## Compute the velocity and duration to go trougth the path

work on one segment of the path :

In [ ]:
from scipy.interpolate import RegularGridInterpolator


def arc_duration(seg, nb_sub=20):
    seg = seg.T
    sub = np.linspace(seg[:, 0], seg[:, 1], nb_sub)
    sub = sub[:-1] + np.diff(sub, axis=0) / 2
    sub_length = np.linalg.norm(np.diff(seg)) / (nb_sub - 1)
    v = world_definition(sub[:, 0], sub[:, 1])
    duration = (sub_length / v).sum()
    return duration, sub, v

In [ ]:
# test for one segment of the path
seg = path[:2, :]
duration, sub, v = arc_duration(seg, nb_sub=20)
duration

Work on the full path :

In [ ]:
total = 0
for i in range(len(path) - 1):
    seg = path[i : i + 2, :]
    duration, sub, v = arc_duration(seg, nb_sub=20)
    total += duration
    print("segment {0} : {1:0.2f}s".format(i, duration))
print("Total   : {0:0.2f}s".format(total))

In [ ]:
def total_duration(path, blabla=False):
    total = 0.0
    for i in range(len(path) - 1):
        seg = path[i : i + 2, :].copy()
        duration, _, _ = arc_duration(seg, nb_sub=20)
        total += duration
        if blabla:
            print("segment {0} : {1:0.1f}s".format(i, duration))
    if blabla:
        print("Total   : {0:0.1f}s".format(total))
    return total


# test for the path
total_duration(path, blabla=True)

### Widget plot

In [ ]:
plt.figure()
(l,) = plt.plot(path[:, 0], path[:, 1], ":o")
plt.contourf(x_world, y_world, v_world, 20, cmap=cm.jet)
title = plt.title("duration")


@ipw.interact(x1=(0.0, 100, 1), x2=(0.0, 100, 1))
def update(x1=30, x2=30):
    path[1, 0] = x1
    path[2, 0] = x2
    t_total = total_duration(path, blabla=False)
    l.set_data(path[:, 0], path[:, 1])
    title.set_text("Total   : {0:0.1f}s".format(t_total))

### What is the optimal solution ?
#### First idea : test all combination 

In [ ]:
Nx, Ny = 50, 50
x1 = np.linspace(5, 80, Nx)
x2 = np.linspace(5, 80, Ny)
X, Y = np.meshgrid(x1, x2)
Z = np.zeros_like(X)
for i in range(len(X)):
    for j in range(len(Y)):
        path[1, 0] = X[i, j]
        path[2, 0] = Y[i, j]
        Z[i, j] = total_duration(path, blabla=False)

In [ ]:
x1_min, x2_min = np.where(np.min(Z) == Z)
X_min = X[x1_min, x2_min]
Y_min = Y[x1_min, x2_min]
print(X_min, Y_min)

In [ ]:
plt.figure()
title = plt.title("")
plt.contourf(X, Y, Z, 20, cmap=cm.jet)
plt.colorbar()
plt.contour(X, Y, Z, 20, cmap=cm.gray)
plt.plot(X_min, Y_min, "or")
plt.show()

### Is that realistic with a finer grid ? (or in larger dimension)

In [ ]:
%%timeit
total_duration(path, blabla=False)

### Usning optimization methode

In [ ]:
def cost(X):
    path[1, 0] = X[0]
    path[2, 0] = X[1]
    return total_duration(path, blabla=False)

In [ ]:
from scipy import optimize

sol = optimize.minimize(cost, [10.0, 10.0], method="Nelder-Mead")
sol

## Crossing a strange river

In [ ]:
# Oscilatting River
def world_definition(x, y):
    v = 1 * np.ones_like(x)
    v = 2 + np.cos(x / 5)
    v[y <= 10] = 10
    v[y > 50] = 10
    return v


x = np.linspace(0, 100, 100)
y = np.linspace(0, 60, 150)
x_world, y_world = np.meshgrid(x, y, indexing="xy")
v_world = world_definition(x_world, y_world)

In [ ]:
# Low velocity
def world_definition(x, y):
    tau = 20
    v = 10 - 5 * np.exp(-(((x - 50) / tau) ** 2) - ((y - 30) / tau) ** 2)
    # v[y<=10]=10
    # v[y>50]=10
    return v


x = np.linspace(0, 100, 100)
y = np.linspace(0, 60, 150)
x_world, y_world = np.meshgrid(x, y, indexing="xy")
v_world = world_definition(x_world, y_world)

In [ ]:
plt.figure()
plt.contourf(x_world, y_world, v_world, 20, cmap=cm.jet)
plt.title("Velocity map")
plt.colorbar()
plt.show()

In [ ]:
Nx, Ny = 50, 50
x1 = np.linspace(5, 80, Nx)
x2 = np.linspace(5, 80, Ny)
X, Y = np.meshgrid(x1, x2)
Z = np.zeros_like(X)
for i in range(len(X)):
    for j in range(len(Y)):
        path[1, 0] = X[i, j]
        path[2, 0] = Y[i, j]
        Z[i, j] = total_duration(path, blabla=False)

In [ ]:
x1_min, x2_min = np.where(np.min(Z) == Z)
X_min = X[x1_min, x2_min]
Y_min = Y[x1_min, x2_min]
print(X_min, Y_min)

In [ ]:
plt.figure()
title = plt.title("")
plt.contourf(X, Y, Z, 20, cmap=cm.jet)
plt.colorbar()
plt.contour(X, Y, Z, 20, cmap=cm.gray)
plt.plot(X_min, Y_min, "or")
plt.show()

In [ ]:
path[1, 0] = X_min
path[2, 0] = Y_min
plt.figure()
plt.contourf(x_world, y_world, v_world, 20, cmap=cm.jet)
plt.plot(path[:, 0], path[:, 1], "o-r")
plt.title("Velocity map")
plt.colorbar()

## Add more control points

In [ ]:
path_init = np.linspace([5, 5], [80, 55], 15)
path = np.copy(path_init)

In [ ]:
def cost_n_points(X):
    for i in range(len(X)):
        path[i + 1, 0] = X[i]
    return total_duration(path, blabla=False)

In [ ]:
from scipy import optimize

init_guess = np.copy(path_init[:-2, 0])
sol = optimize.minimize(cost_n_points, init_guess, method="Nelder-Mead")
X = sol.x
for i in range(len(X)):
    path[i + 1, 0] = X[i]

In [ ]:
sol

In [ ]:
plt.figure()
plt.contourf(x_world, y_world, v_world, 20, cmap=cm.jet)
plt.plot(path_init[:, 0], path_init[:, 1], "o:g")
plt.plot(path[:, 0], path[:, 1], "o-r")
plt.title("Velocity map")
plt.colorbar()